In [6]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50  # Or your specific ResNet variant
from PIL import Image
import numpy as np
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

In [5]:
pip install grad-cam

     ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
     ---------- ----------------------------- 2.1/7.8 MB 14.7 MB/s eta 0:00:01
     ---------------- ----------------------- 3.1/7.8 MB 16.8 MB/s eta 0:00:01
     ----------------------------- ---------- 5.8/7.8 MB 10.1 MB/s eta 0:00:01
     ----------------------------------- ---- 6.8/7.8 MB 8.7 MB/s eta 0:00:01
     ---------------------------------------  7.6/7.8 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------- 7.8/7.8 MB 6.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44339 sha256=d6c70c8d8f159a9724e33ff689ac7bb4676ae2f

In [7]:
# --- 1. Define Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [8]:
# --- 2. Instantiate Your ResNet Model Architecture ---
model = resnet50(pretrained=False)

c:\Users\LAPZONE\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LAPZONE\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# --- 3. Load the State Dictionary from Your .pth File ---
pth_file_path = 'C:/Users/LAPZONE/OneDrive/Desktop/MIET/Gradcam/plant_disease_model.pth'  # Replace with the actual path to your .pth file
try:
    checkpoint = torch.load(pth_file_path, map_location=device)
    model.load_state_dict(checkpoint)
except FileNotFoundError:
    print(f"Error: Model file not found at {pth_file_path}")
    exit()
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

Error loading model: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.0.conv3.weight", "layer1.0.bn3.weight", "layer1.0.bn3.bias", "layer1.0.bn3.running_mean", "layer1.0.bn3.running_var", "layer1.0.downsample.0.weight", "layer1.0.downsample.1.weight", "layer1.0.downsample.1.bias", "layer1.0.downsample.1.running_mean", "layer1.0.downsample.1.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer1.1.conv3.weight", 

C:\Users\LAPZONE\AppData\Local\Temp\ipykernel_17068\1223694782.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(pth_file_path, map_location=device

: 

In [1]:
import torch
# --- 1. Define Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torchvision.models import resnet50
# --- 2. Instantiate Your ResNet Model Architecture ---
model = resnet50(pretrained=False)
# --- 4. Set Model to Evaluation Mode ---
model.eval().to(device)

c:\Users\LAPZONE\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LAPZONE\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [2]:
import torchvision.transforms as transforms

In [3]:
# --- 5. Define Image Transformations (Match Your Training) ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust if your input size was different
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization if applicable
])

In [4]:
from PIL import Image

In [6]:
import torch
# --- 1. Define Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torchvision.transforms as transforms
from PIL import Image
# --- 5. Define Image Transformations (Match Your Training) ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust if your input size was different
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization if applicable
])
# --- 6. Load and Preprocess the Image for Explanation ---
image_path = 'C:/Users/LAPZONE/OneDrive/Desktop/MIET/Gradcam/test.jpeg'  # Replace with the path to your image
try:
    original_image = Image.open(image_path).convert('RGB')
    input_tensor = transform(original_image).unsqueeze(0).to(device)  # Add batch dimension
except FileNotFoundError:
    print(f"Error: Image not found at {image_path}")
    exit()
except Exception as e:
    print(f"Error loading image: {e}")
    exit()

In [7]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50  # Or your specific ResNet variant
from PIL import Image
import numpy as np
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
# --- 2. Instantiate Your ResNet Model Architecture ---
model = resnet50(pretrained=False)
target_layer = model.layer4[-1].conv3  # Try this for ResNet50 initially


c:\Users\LAPZONE\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LAPZONE\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
cam = GradCAM(model=model, target_layers=[target_layer])

In [9]:
# --- 9. Get Model Prediction (Optional, but useful for targeting a specific class) ---
output = model(input_tensor)
_, predicted_class = torch.max(output, 1)
predicted_class_index = predicted_class.item()
print(f"Predicted class index: {predicted_class_index}")

Predicted class index: 968


In [10]:
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

In [11]:
# --- 10. Generate the Grad-CAM Heatmap ---
# You can target a specific class or use the predicted class
targets = [ClassifierOutputTarget(predicted_class_index)]  # Create the targets list
# specific_target_class_index = your_target_class_index
# targets = [ClassifierOutputTarget(specific_target_class_index)]

grayscale_cam = cam(input_tensor=input_tensor, targets=targets)  # Use 'targets'
grayscale_cam = grayscale_cam[0, :]  # Remove batch dimension

In [12]:
# --- 11. Visualize the Heatmap on the Original Image ---
original_image_np = np.array(original_image)[:, :, ::-1]  # Convert RGB to BGR for OpenCV

# Resize the grayscale_cam to the original image size
resized_cam = cv2.resize(grayscale_cam, (original_image_np.shape[1], original_image_np.shape[0]))

visualization = show_cam_on_image(original_image_np / 255.0, resized_cam, use_rgb=True)

In [13]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50  # Or your specific ResNet variant
from PIL import Image
import numpy as np
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# --- 11. Visualize the Heatmap on the Original Image ---
original_image_np = np.array(original_image)[:, :, ::-1]  # Convert RGB to BGR for OpenCV
resized_cam = cv2.resize(grayscale_cam, (original_image_np.shape[1], original_image_np.shape[0]))
visualization = show_cam_on_image(original_image_np / 255.0, resized_cam, use_rgb=True)

import cv2
# --- 12. Display or Save the Visualization ---
cv2.imshow("Grad-CAM Visualization", visualization)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imwrite("gradcam_output.jpg", visualization)

# --- 13. Clean Up ---
cam.remove_hook()

AttributeError: 'GradCAM' object has no attribute 'remove_hook'